In [143]:
# %pip install -U pip wheel
# %pip install -r ../requirements.txt

In [144]:
import numpy as np
import pandas as pd
import importlib

import ipywidgets as widgets


In [145]:
import cpagent
import cprender
import cpenvs
import cppid

_ = importlib.reload(cppid)
_ = importlib.reload(cpagent)
_ = importlib.reload(cprender)
_ = importlib.reload(cpenvs)

In [146]:
agent = cppid.PidAgent((0.1, 0.5, 0.02), (0.01, 0.0, 0.00))
df = cpagent.execute_cartpole(agent, env=cpenvs.MovingCartpoleEnv())
df

,ep,t,cart_pos,cart_vel,pole_ang,pole_vel,pos_deviation,reward,cart_pos_setpoint
0,0,0,-0.006284,0.028929,-0.001937,-0.021673,-0.006284,NaN,0.0
1,0,1,-0.005705,-0.166165,-0.002370,0.270398,-0.005705,0.999984,0.0
2,0,2,-0.009028,-0.361253,0.003038,0.562333,-0.009028,0.999959,0.0
3,0,3,-0.016253,-0.166174,0.014285,0.270608,-0.016253,0.999868,0.0
4,0,4,-0.019577,0.028742,0.019697,-0.017535,-0.019577,0.999808,0.0
...,...,...,...,...,...,...,...,...,...
10015,19,496,-0.042066,-0.008046,-0.010762,0.032627,-0.042066,0.999115,0.0
10016,19,497,-0.042227,-0.203012,-0.010110,0.321895,-0.042227,0.999108,0.0
10017,19,498,-0.046287,-0.007747,-0.003672,0.026041,-0.046287,0.998929,0.0
10018,19,499,-0.046442,0.187427,-0.003151,-0.267799,-0.046442,0.998922,0.0


In [147]:
_ = widgets.interact(
    lambda ep, t: cprender.render_cartpole_state_df(df, t, ep),
    ep=widgets.IntSlider(min=0, max=20, step=1, value=0),
    t=widgets.IntSlider(min=0, max=500, step=1, value=0),
)

interactive(children=(IntSlider(value=0, description='ep', max=20), IntSlider(value=0, description='t', max=50…

In [148]:
EP = 0

fig = cprender.lineplot(df, ep=EP, incl_velo=False)
fig.show()